In [10]:
from IPython.display import clear_output
from copy import deepcopy
from math import inf

In [11]:
def display_board(board, turn=None, show_labels=False, selected_row=None, ai=False):
    if show_labels:
        print(end=' ')
        for i in range(1, len(board[0]) + 1):
            print(i, end='    ' if i < 9 else '   ')
        print('\n')
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == [1]:
                symbol = '[B]'
            elif board[i][j] == [4]:
                symbol = '[Y]'
            elif board[i][j] == [7]:
                symbol = '[R]'
            elif board[i][j] == [10]:
                symbol = '[G]'
            elif i == selected_row:
                symbol = '[X]'
#             elif board[i][j] == []:
#                 symbol = '[ ]'
            else:
                symbol = '[ ]'
#                 symbol = board[i][j]
            print(symbol, end='  ')
        if show_labels:
            print(i + 1)
        print('\n')
    if turn != None:
        print(f"{['B', 'Y', 'R', 'G'][turn]}'s turn")
        if ai:
            print("(AI)")

def rotate_piece(piece, rotation):
    for i in range(rotation):
        piece = tuple(zip(*piece[::-1]))
    return piece

def display_piece(piece, show_unit_numbers=False, selected_unit_number=0):
    unit_number = 0
    for i in range(len(piece)):
        if 1 in piece[i]:
            for unit in piece[i][1:]:
                if unit == 1:
                    unit_number += 1
                    if unit_number == selected_unit_number:
                        symbol = '[X]'
                    elif show_unit_numbers:
                        symbol = [unit_number]
                    else:
                        symbol = '[ ]'
                else:
                    symbol = '   '
                print(symbol, end='  ')
            if i != len(piece)-1:
                print('\n')
    return unit_number

def valid_move(board, pieces, turn, move):
    piece = rotate_piece(pieces[turn][move[1][0]], move[1][1])
    unit_number = 0
    touching_corner = False
    for i in range(len(piece)):
        for j in range(len(piece[i])):
            if piece[i][j] == 1:
                unit_number += 1
                if unit_number == move[1][2]:
                    for k in range(len(piece)):
                        for l in range(len(piece[k])):
                            if piece[k][l] == 1:
                                m = move[0][0]+(k-i)
                                n = move[0][1]+(l-j)
                                if m < 0 or m > 19 or n < 0 or n > 19:
                                    return False
                                for o in range(4):
                                    if o * 3 + 1 in board[m][n]:
                                        return False
                                if turn * 3 + 3 in board[m][n]:
                                    return False
                                if turn * 3 + 2 in board[m][n]:
                                    touching_corner = True
                    return True if touching_corner else False

def has_pieces(pieces, turn):
    for piece in pieces[turn]:
        if piece:
            return True
    return False

def get_valid_moves(board, pieces, turn, only_one_move=False, max_pieces=None):
    if max_pieces != None:
        pieces_count = 0
    valid_moves = []
    valid_boards = []
    for i in range(len(board)):
        for j in range(len(board[i])):
            if turn * 3 + 2 in board[i][j]:
                for k in range(len(pieces[turn])) if max_pieces == None else range(len(pieces[turn]) - 1, -1, -1):
                    if max_pieces != None:
                        if pieces_count == max_pieces:
                            return valid_moves
                        pieces_count += 1
                    if pieces[turn][k]:
                        rotated_pieces = []
                        for rotation in range(4):
                            rotated_piece = rotate_piece(deepcopy(pieces)[turn][k], rotation)
                            if rotated_piece not in rotated_pieces:
                                rotated_pieces.append(rotated_piece)
                                if k != 0:
                                    unit_number = 0
                                    for row in pieces[turn][k]:
                                        for unit in row:
                                            if unit == 1:
                                                unit_number += 1
                                                move = ((i, j), (k, rotation, unit_number))
                                                if valid_move(board, deepcopy(pieces), turn, move):
                                                    board_2 = deepcopy(board)
                                                    play_move(board_2, deepcopy(pieces), turn, move)
                                                    if board_2 not in valid_boards:
                                                        if only_one_move:
                                                            return True
                                                        valid_moves.append(move)
                                                        valid_boards.append(board_2)
                                else:
                                    move = ((i, j), (k, rotation, 1))
                                    if valid_move(board, deepcopy(pieces), turn, move):
                                        board_2 = deepcopy(board)
                                        play_move(board_2, deepcopy(pieces), turn, move)
                                        if board_2 not in valid_boards:
                                            if only_one_move:
                                                return True
                                            valid_moves.append(move)
                                            valid_boards.append(board_2)
    return False if only_one_move else valid_moves

def get_max_depth(pieces, turn):
    pass

def get_score(pieces, player, played_dot_last):
    if not has_pieces(pieces, player):
        return 20 if played_dot_last[player] else 15
    score = 0
    for i in range(len(pieces[player])):
        if pieces[player][i]:
            if i == 0:
                score -= 1
            elif i == 1:
                score -= 2
            elif i <= 3:
                score -= 3
            elif i <= 8:
                score -= 4
            else:
                score -= 5
    return score

def evaluate(board, pieces, turn, maximizing_player, players_done, played_dot_last):
    evaluation = 0
    for player in range(4):
        evaluation += get_score(pieces, player, played_dot_last) if player == maximizing_player else -get_score(pieces, player, played_dot_last)
        valid_moves = get_valid_moves(deepcopy(board), deepcopy(pieces), player)
        valid_moves_evaluation = 0
        for move in valid_moves:
            if move[1][0] == 0:
                valid_moves_evaluation += 0.2
            elif move[1][0] == 1:
                valid_moves_evaluation += 0.4
            elif move[1][0] <= 3:
                valid_moves_evaluation += 0.6
            elif move[1][0] <= 8:
                valid_moves_evaluation += 0.8
            else:
                valid_moves_evaluation += 1
        evaluation += valid_moves_evaluation if player == maximizing_player else -valid_moves_evaluation
        if pieces[player][0]:
            evaluation += 5 if player == maximizing_player else -5
    return evaluation if turn == maximizing_player else -evaluation

def minimax(board, pieces, turn, depth, alpha, beta, maximizing_player, players_done, played_dot_last, max_depth):
    if not has_pieces(pieces, turn) and not get_valid_moves(deepcopy(board), deepcopy(pieces), turn, True) and turn not in players_done:
        players_done.append(turn)
    if depth == max_depth or len(players_done) == 4:
        return evaluate(board, pieces, turn, maximizing_player, players_done, played_dot_last)
    if depth != 0:
        turn = 0 if turn == 3 else turn + 1
    if turn == maximizing_player:
        max_evaluation = -inf
        for move in get_valid_moves(deepcopy(board), deepcopy(pieces), turn, max_pieces=12):
            board_2 = deepcopy(board)
            had_dot = pieces[turn][0] != ()
            play_move(board_2, deepcopy(pieces), turn, move)
            if not has_pieces(pieces, turn) and had_dot:
                played_dot_last[turn] = True
            evaluation = round(minimax(board_2, pieces, turn, depth + 1, alpha, beta, maximizing_player, players_done[:], played_dot_last[:], max_depth), 1)
            if evaluation > max_evaluation:
                max_evaluation = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        if depth == 0:
            print(f'({round(max_evaluation, 2)})')
            try:
                return best_move
            except:
                return move
        return max_evaluation
    min_evaluation = inf
    for move in get_valid_moves(deepcopy(board), deepcopy(pieces), turn, max_pieces=12):
        board_2 = deepcopy(board)
        had_dot = pieces[turn][0] != ()
        play_move(board_2, deepcopy(pieces), turn, move)
        if not has_pieces(pieces, turn) and had_dot:
            played_dot_last[turn] = True
        evaluation = round(minimax(board_2, pieces, turn, depth + 1, alpha, beta, maximizing_player, players_done[:], played_dot_last[:], max_depth), 1)
        min_evaluation = min(min_evaluation, evaluation)
        beta = min(beta, evaluation)
        if beta <= alpha:
            break
    return min_evaluation

def play_move(board, pieces, turn, move):
    piece = rotate_piece(pieces[turn][move[1][0]], move[1][1])
    unit_number = 0
    for i in range(len(piece)):
        for j in range(len(piece[i])):
            if piece[i][j] == 1:
                unit_number += 1
                if unit_number == move[1][2]:
                    for k in range(len(piece)):
                        for l in range(len(piece[k])):
                            m = move[0][0]+(k-i)
                            n = move[0][1]+(l-j)
                            if piece[k][l] == 1:
                                board[m][n] = [1 + turn * 3]
                            elif piece[k][l] != 0 and m >= 0 and m <= 19 and n >= 0 and n <= 19:
                                if piece[k][l] == 3 and 2 + turn * 3 in board[m][n]:
                                    board[m][n].remove(2 + turn * 3)
                                if not board[m][n] or (board[m][n][0] - 1) % 3 != 0 and piece[k][l] + turn * 3 not in board[m][n] and 3 + turn * 3 not in board[m][n]:
                                    board[m][n].append(piece[k][l] + turn * 3)
                    pieces[turn][move[1][0]] = ()
                    return

def play_blokus(ai=[2, 3, 4], max_depth=0):
    board = [[[] for i in range(20)] for i in range(20)]
    board[0][0] = [2]
    board[0][-1] = [5]
    board[-1][-1] = [8]
    board[-1][0] = [11]
    pieces = [[((2, 3, 2),
                (3, 1, 3),
                (2, 3, 2)),
               
               ((2, 3, 2),
                (3, 1, 3),
                (3, 1, 3),
                (2, 3, 2)),
              
               ((2, 3, 2),
                (3, 1, 3),
                (3, 1, 3),
                (3, 1, 3),
                (2, 3, 2)),
              
               ((2, 3, 2, 0),
                (3, 1, 3, 2),
                (3, 1, 1, 3),
                (2, 3, 3, 2)),
              
               ((2, 3, 2),
                (3, 1, 3),
                (3, 1, 3),
                (3, 1, 3),
                (3, 1, 3),
                (2, 3, 2)),
              
               ((0, 2, 3, 2),
                (0, 3, 1, 3),
                (2, 3, 1, 3),
                (3, 1, 1, 3),
                (2, 3, 3, 2)),
              
               ((2, 3, 2, 0),
                (3, 1, 3, 2),
                (3, 1, 1, 3),
                (3, 1, 3, 2),
                (2, 3, 2, 0)),
              
               ((2, 3, 3, 2),
                (3, 1, 1, 3),
                (3, 1, 1, 3),
                (2, 3, 3, 2)),
              
               ((2, 3, 3, 2, 0),
                (3, 1, 1, 3, 2),
                (2, 3, 1, 1, 3),
                (0, 2, 3, 3, 2)),
              
               ((2, 3, 2),
                (3, 1, 3),
                (3, 1, 3),
                (3, 1, 3),
                (3, 1, 3),
                (3, 1, 3),
                (2, 3, 2)),
              
               ((0, 2, 3, 2),
                (0, 3, 1, 3),
                (0, 3, 1, 3),
                (2, 3, 1, 3),
                (3, 1, 1, 3),
                (2, 3, 3, 2)),
              
               ((0, 2, 3, 2),
                (0, 3, 1, 3),
                (2, 3, 1, 3),
                (3, 1, 1, 3),
                (3, 1, 3, 2),
                (2, 3, 2, 0)),
              
               ((0, 2, 3, 2),
                (2, 3, 1, 3),
                (3, 1, 1, 3),
                (3, 1, 1, 3),
                (2, 3, 3, 2)),
              
               ((2, 3, 3, 2),
                (3, 1, 1, 3),
                (2, 3, 1, 3),
                (3, 1, 1, 3),
                (2, 3, 3, 2)),
              
               ((2, 3, 2, 0),
                (3, 1, 3, 2),
                (3, 1, 1, 3),
                (3, 1, 3, 2),
                (3, 1, 3, 0),
                (2, 3, 2, 0)),
              
               ((0, 2, 3, 2, 0),
                (0, 3, 1, 3, 0),
                (2, 3, 1, 3, 2),
                (3, 1, 1, 1, 3),
                (2, 3, 3, 3, 2)),
              
               ((2, 3, 2, 0, 0),
                (3, 1, 3, 0, 0),
                (3, 1, 3, 3, 2),
                (3, 1, 1, 1, 3),
                (2, 3, 3, 3, 2)),
              
               ((2, 3, 3, 2, 0),
                (3, 1, 1, 3, 2),
                (2, 3, 1, 1, 3),
                (0, 2, 3, 1, 3),
                (0, 0, 2, 3, 2)),
              
               ((2, 3, 2, 0, 0),
                (3, 1, 3, 3, 2),
                (3, 1, 1, 1, 3),
                (2, 3, 3, 1, 3),
                (0, 0, 2, 3, 2)),
              
               ((2, 3, 2, 0, 0),
                (3, 1, 3, 3, 2),
                (3, 1, 1, 1, 3),
                (2, 3, 1, 3, 2),
                (0, 2, 3, 2, 0)),
              
               ((0, 2, 3, 2, 0),
                (2, 3, 1, 3, 2),
                (3, 1, 1, 1, 3),
                (2, 3, 1, 3, 2),
                (0, 2, 3, 2, 0))] for i in range(4)]
    played_dot_last = [False] * 4
    players_done = []
    turn = 0
    while len(players_done) != 4:
        if has_pieces(pieces, turn) and get_valid_moves(deepcopy(board), deepcopy(pieces), turn, True):
            if turn + 1 in ai:
                clear_output()
                display_board(board, turn, ai=True)
                had_dot = pieces[turn][0] != ()
                if max_depth == 0:
                    max_depth = get_max_depth(pieces, turn)
                move = minimax(board, pieces, turn, 0, -inf, inf, turn, players_done[:], played_dot_last[:], max_depth)
                print(move)
                play_move(board, pieces, turn, move)
                if not has_pieces(pieces, turn) and had_dot:
                    played_dot_last[turn] = True
                clear_output()
                display_board(board, turn, ai=True)
            else:
                while True:
                    while True:
                        clear_output()
                        display_board(board, turn)
                        try:
                            k = input('\npiece: ')
                            if k == 'end':
                                clear_output()
                                return
                            k = int(k) - 1
                            if k >= 0 and k <= 20 and pieces[turn][k]:
                                piece = pieces[turn][k]
                                break
                        except:
                            continue
                    while True:
                        clear_output()
                        display_board(board, turn)
                        rotations = {}
                        for rotation in range(4):
                            rotated_piece = rotate_piece(piece, rotation)
                            if rotated_piece not in rotations.values():
                                rotations[rotation] = rotated_piece
                        if len(rotations) > 1:
                            for rotation in rotations:
                                print(f'\n{rotation + 1})\n')
                                display_piece(rotations[rotation])
                            try:
                                rotation = input('\nrotation: ')
                                if rotation == 'end':
                                    clear_output()
                                    return
                                rotation = int(rotation) - 1
                                if rotation in rotations:
                                    piece = rotate_piece(piece, rotation)
                                    break
                            except:
                                continue
                        else:
                            rotation = 0
                            break
                    if k != 0:
                        while True:
                            clear_output()
                            display_board(board, turn)
                            print()
                            max_unit_number = display_piece(piece, True)
                            try:
                                unit_number = input('\nunit number: ')
                                if unit_number == 'end':
                                    clear_output()
                                    return
                                unit_number = int(unit_number)
                                if unit_number >= 1 and unit_number <= max_unit_number:
                                    break
                            except:
                                continue
                    else:
                        unit_number = 1
                    while True:
                        clear_output()
                        display_board(board, turn, True)
                        print()
                        display_piece(piece, False, unit_number)
                        try:
                            i = input('\nrow: ')
                            if i == 'end':
                                clear_output()
                                return
                            i = int(i) - 1
                            if i >= 0 and i <= 19:
                                break
                        except:
                            continue
                    while True:
                        clear_output()
                        display_board(board, turn, True, i)
                        print()
                        display_piece(piece, False, unit_number)
                        try:
                            j = input('\ncolumn: ')
                            if j == 'end':
                                clear_output()
                                return
                            j = int(j) - 1
                            if j >= 0 and j <= 19:
                                break
                        except:
                            continue
                    move = ((i, j), (k, rotation, unit_number))
                    if valid_move(board, deepcopy(pieces), turn, move):
                        break
                play_move(board, pieces, turn, move)
                if not has_pieces(pieces, turn) and k == 0:
                    played_dot_last[turn] = True
        elif turn not in players_done:
            players_done.append(turn)
        turn = 0 if turn == 3 else turn + 1
    scores = {['B', 'Y', 'R', 'G'][player]: get_score(pieces, player, played_dot_last) for player in range(4)}
    clear_output()
    display_board(board)
    print(f'game over!\n\nfinal score: {scores}')

In [12]:
play_blokus(ai=[1, 2, 3, 4], max_depth=1)

[B]  [B]  [B]  [B]  [ ]  [B]  [B]  [B]  [ ]  [B]  [B]  [Y]  [Y]  [Y]  [Y]  [Y]  [ ]  [ ]  [ ]  [Y]  

[ ]  [ ]  [B]  [ ]  [B]  [ ]  [ ]  [B]  [B]  [ ]  [B]  [B]  [ ]  [ ]  [ ]  [ ]  [Y]  [Y]  [ ]  [Y]  

[B]  [B]  [ ]  [ ]  [B]  [B]  [B]  [ ]  [ ]  [ ]  [ ]  [B]  [Y]  [Y]  [ ]  [Y]  [Y]  [ ]  [Y]  [Y]  

[B]  [B]  [ ]  [ ]  [ ]  [B]  [ ]  [ ]  [ ]  [ ]  [ ]  [Y]  [Y]  [ ]  [Y]  [ ]  [Y]  [ ]  [ ]  [Y]  

[B]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [Y]  [ ]  [Y]  [Y]  [Y]  [ ]  [ ]  [ ]  [ ]  

[ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [Y]  [ ]  [ ]  [ ]  [ ]  [ ]  

[ ]  [G]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  

[ ]  [G]  [G]  [G]  [ ]  [ ]  [ ]  [ ]  [R]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  

[ ]  [G]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [R]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  

[ ]  [ ]  [G]  [ ]  [ ]  [ ]  [ ]  [R]  [R]  [R]  [ ]  [ ]  [ ]  [ ]  [ ]  [ ]  [ 

UnboundLocalError: local variable 'move' referenced before assignment